# Pre set up

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [2]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 13,13
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()

# Run a pre-trained detectron2 model

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("./input.jpg")

In [ ]:
imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("amodal_train",)
cfg.DATASETS.TEST = ("small_amodal_test")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
outputs["instances"].pred_classes
outputs["instances"].pred_boxes

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

In [ ]:
imshow(v.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("amodal_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "amodal_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Train pre-trained model on a custom dataset (FT)

## Prepare the dataset

In [3]:
# Register my amodal datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_coco_train", {}, "datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json", "datasets/coco/train2014")
# Prepare test datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_coco_val", {}, "datasets/coco/annotations/COCO_amodal_val2014_with_classes_poly.json", "datasets/coco/val2014")
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("amodal_coco_train")

[02/28 23:05:57 d2.data.datasets.coco]: Loaded 2276 images in COCO format from datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json


In [ ]:
# Register small datasets for debugging 
from detectron2.data import DatasetCatalog,MetadataCatalog
dataset_dicts = DatasetCatalog.get("amodal_train")
import random
DatasetCatalog.register("small_amodal_test", lambda : random.sample(dataset_dicts,k=4))
metadata = {}
MetadataCatalog.get("small_amodal_test").set(
        json_file="datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json", image_root="datasets/coco/train2014", evaluator_type="coco", **metadata
    )


In [ ]:
print(dataset_dicts[3]['annotations'][4]['invisible_mask'])
print(dataset_dicts[3]['annotations'][2]['visible_mask'])
print(dataset_dicts[3]['annotations'][2]['segmentation'])
MetadataCatalog.get("amodal_train")

In [ ]:
 # Run pre-trained model on this amodal image 
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
from detectron2.data import DatasetMapper
mapper = DatasetMapper(cfg,is_train=True)
mydict = mapper(dataset_dicts[3])
boxes = mydict['instances'].gt_boxes.tensor
print(mydict['instances'].gt_masks)

In [ ]:
## To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
dataset_dicts = DatasetCatalog.get("amodal_train")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("amodal_train"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    imshow(vis.get_image()[:, :, ::-1])
    predictor = DefaultPredictor(cfg)
    outputs = predictor(img)
    # We can use `Visualizer` to draw the predictions on the image.
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get("amodal_train"), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(v.get_image()[:, :, ::-1])
    
    

## Train 

In [ ]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.DATASETS.TEST = ("amodal_coco_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.STEPS = (1200,1300)
cfg.SOLVER.MAX_ITER = 1500
cfg.VIS_PERIOD = 20
cfg.OUTPUT_DIR = "myAmodalCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[02/28 23:06:18 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[02/28 23:06:18 d2.data.datasets.coco]: Loaded 2276 images in COCO format from datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json
[02/28 23:06:18 d2.data.build]: Removed 0 images with no usable annotations. 2276 images left.
[02/28 23:06:18 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 2311         |   bicycle    | 21           |      car      | 279          |
|  motorcycle   | 47           |   airplane   | 65           |      bus      | 92           |
|     train     | 60           |    truck     | 115          |     boat      | 79           |
| traffic light | 13           | fire hydrant | 35           |   stop sign   | 12           |
| parking meter | 10           |    bench     | 48           |     bird      | 57           |
|      cat    

[02/28 23:08:17 d2.utils.events]: eta: 0:04:50  iter: 399  total_loss: 0.533  loss_cls: 0.123  loss_box_reg: 0.125  loss_mask: 0.221  loss_rpn_cls: 0.010  loss_rpn_loc: 0.010  time: 0.2752  data_time: 0.0145  lr: 0.000200  max_mem: 2922M
[02/28 23:08:23 d2.utils.events]: eta: 0:04:45  iter: 419  total_loss: 0.428  loss_cls: 0.089  loss_box_reg: 0.091  loss_mask: 0.208  loss_rpn_cls: 0.008  loss_rpn_loc: 0.013  time: 0.2755  data_time: 0.0192  lr: 0.000210  max_mem: 2922M
[02/28 23:08:30 d2.utils.events]: eta: 0:04:40  iter: 439  total_loss: 0.460  loss_cls: 0.109  loss_box_reg: 0.096  loss_mask: 0.211  loss_rpn_cls: 0.009  loss_rpn_loc: 0.011  time: 0.2761  data_time: 0.0135  lr: 0.000220  max_mem: 2922M
[02/28 23:08:35 d2.utils.events]: eta: 0:04:34  iter: 459  total_loss: 0.347  loss_cls: 0.095  loss_box_reg: 0.073  loss_mask: 0.157  loss_rpn_cls: 0.006  loss_rpn_loc: 0.008  time: 0.2761  data_time: 0.0119  lr: 0.000230  max_mem: 2922M
[02/28 23:08:42 d2.utils.events]: eta: 0:04:29  

In [ ]:
!tensorboard --logdir=myAmodalCheckpoint --port=6006

##  Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:

In [ ]:
import os
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.OUTPUT_DIR = "myAmodalCheckpoint"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.WEIGHTS =  model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)

In [ ]:
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("amodal_coco_val", cfg, False, output_dir="myAmodalEvaluation")
val_loader = build_detection_test_loader(cfg, "amodal_coco_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get("amodal_coco_val")
for d in random.sample(dataset_dicts, 2):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("amodal_coco_val"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(v.get_image()[:, :, ::-1])